按顺序提取json文件中的answers和question标签，并且直接输出 answers 和 question 的内容

In [ ]:
import json

# 文件路径列表
file_paths = [
    ("AAT.json", r"./【AiDLab】A100/AAT/label/AAT.json"),
    ("LAT.json", r"./【AiDLab】A100/LAT/label/LAT.json")
]

# 用于存储所有的 answers 和 questions
all_answers = []
all_questions = []

# 遍历文件路径，加载并提取每个文件中的标签
for file_name, file_path in file_paths:
    # 加载 JSON 文件
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    # 顺序提取 answers 和 questions，并添加到对应列表中
    for item in data:
        all_answers.extend(item.get('answers', []))
        all_questions.extend(item.get('question', []))

# 打印汇总后的标签数量
print(f"总共提取到 {len(all_answers)} 个 answers 标签。")
print(f"总共提取到 {len(all_questions)} 个 questions 标签。")
print()

# 打印汇总后的标签
print("提取到的所有 answers 标签：")
for answer in all_answers:
    print(f"  - {answer}")

print("\n提取到的所有 questions 标签：")
for question in all_questions:
    print(f"  - {question}")


提取每个 answers 和 questions 字符串中的编号，并使用这个编号来匹配数据集中的图片文件名

In [ ]:
import json
import re

# 文件路径列表
file_paths = [
    ("AAT.json", r"./【AiDLab】A100/AAT/label/AAT.json"),
    ("LAT.json", r"./【AiDLab】A100/LAT/label/LAT.json")
]

# 用于存储文件名与标签的字典
filename_to_label = {}

# 正则表达式提取 AAT 和 LAT 格式的编号
pattern_aat = re.compile(r'(\d{3}A\d{2})')  # AAT 格式，例如 "001A01"
pattern_lat = re.compile(r'P\d{8}')         # LAT 格式，例如 "P00447524"

# 遍历文件路径，加载并提取每个文件中的标签
for file_name, file_path in file_paths:
    # 加载 JSON 文件
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    # 处理 AAT 和 LAT 文件
    for item in data:
        answers = item.get('answers', [])
        questions = item.get('question', [])
        
        # 合并 answers 和 questions 标签
        for label in answers + questions:
            # 匹配 AAT 格式编号
            match_aat = pattern_aat.search(label)
            if match_aat:
                file_id = match_aat.group(1)  # 提取编号部分，例如 "001A01"
                filename_to_label[file_id] = label  # 将编号映射到标签
            
            # 匹配 LAT 格式编号
            match_lat = pattern_lat.search(label)
            if match_lat:
                file_id = match_lat.group(0)  # 提取编号部分，例如 "P00447524"
                filename_to_label[file_id] = label  # 将编号映射到标签

print(f'字典个数{len(filename_to_label)}')
print()
# 输出结果示例
for filename, label in filename_to_label.items():
    print(f"{filename}.jpg: {label}")


先统计一下主类别一共有几类

In [ ]:
import json

# 文件路径列表
file_paths = [
    ("AAT.json", r"./【AiDLab】A100/AAT/label/AAT.json"),
    ("LAT.json", r"./【AiDLab】A100/LAT/label/LAT.json")
]

# 类别映射字典，将标签映射为四大类
category_mapping = {
    'Shoes': 'shoes',
    'Top': 'clothings',
    'Pants': 'clothings',
    'Skirt': 'clothings',
    'Dress': 'clothings',
    'Outwear': 'clothings',
    'Earing': 'accessories',
    'Bracelet': 'accessories',
    'Watches': 'accessories',
    'Hat': 'accessories',
    'Neckline': 'accessories',
    'Sunglasses': 'accessories',
    'Bags': 'bags'
}

# 用于存储主类别的集合
main_categories = set()

# 遍历文件路径，加载并提取主类别
for file_name, file_path in file_paths:
    # 加载 JSON 文件
    with open(file_path, 'r') as f:
        data = json.load(f)

    # 提取 answers 和 questions 中的主类别
    for item in data:
        answers = item.get('answers', [])
        questions = item.get('question', [])
        
        # 合并 answers 和 questions 标签
        for label in answers + questions:
            if "AAT" in file_name:
                # 对 AAT 标签格式进行处理，取主类别部分
                main_category = label.split('/')[0].lower()  # 转换为小写
            elif "LAT" in file_name:
                # 对 LAT 标签格式进行处理，取前缀并通过字典映射
                main_category = label.split('_')[0]
                main_category = category_mapping.get(main_category, None)

                # 如果映射成功，将类别转换为小写
                if main_category:
                    main_category = main_category.lower()

            # 如果主类别存在，将其添加到集合中
            if main_category:
                main_categories.add(main_category)

# 将主类别集合转换为有序列表
labellist = sorted(list(main_categories))

# 输出主类别种类数和类别名称
print(f"主类别总数: {len(main_categories)}")
print("主类别列表:", labellist)


查看一张图片和标签

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import random
import json
import os

# 定义图像转换
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 调整图像大小
    transforms.ToTensor()           # 转换为 Tensor
])

# 统一的类别映射列表，保证 AAT 和 LAT 数据集的一致性
labellist = ['shoes', 'clothing', 'bags', 'accessories']

# 类别映射字典，将 LAT 标签映射到四大类
category_mapping = {
    'Shoes': 'shoes',
    'Top': 'clothing',
    'Pants': 'clothing',
    'Skirt': 'clothing',
    'Dress': 'clothing',
    'Outwear': 'clothing',
    'Earing': 'accessories',
    'Bracelet': 'accessories',
    'Watches': 'accessories',
    'Hat': 'accessories',
    'Neckline': 'accessories',
    'Sunglasses': 'accessories',
    'Bags': 'bags'
}

# 自定义数据集类，包含 AAT 和 LAT 数据集
class FashionDataset(Dataset):
    def __init__(self, images_dirs, json_paths, transform=None):
        self.images_dirs = images_dirs
        self.transform = transform
        self.data = []

        # 加载 AAT 和 LAT 数据
        for images_dir, json_path in zip(images_dirs, json_paths):
            with open(json_path, 'r') as f:
                json_data = json.load(f)
                for item in json_data:
                    # 提取 answers 标签和 image ID
                    answers = item.get('answers', [])
                    if not answers:
                        continue
                    label_str = answers[0]  # 取第一个作为标签
                    
                    # 判断是否为 AAT 或 LAT，并提取标签
                    if 'AAT' in json_path:
                        main_category = label_str.split('/')[0].lower()
                    elif 'LAT' in json_path:
                        main_category = label_str.split('_')[0]
                        main_category = category_mapping.get(main_category, '').lower()

                    # 如果主类别存在于映射列表，则添加到数据集中
                    if main_category in labellist:
                        image_id = label_str.split('_')[-1] + ".jpg"  # 获取文件名
                        img_path = os.path.join(images_dir, image_id)
                        self.data.append((img_path, main_category))

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path, main_category = self.data[idx]
        label = labellist.index(main_category)  # 将主类别转为索引

        # 读取图像
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        
        return image, label

# 数据加载，处理 AAT 和 LAT 文件夹
images_dirs = [
    "./【AiDLab】A100/AAT/image",  # 修改为 AAT 图片文件夹路径
    "./【AiDLab】A100/LAT/image"   # 修改为 LAT 图片文件夹路径
]
json_paths = [
    "./【AiDLab】A100/AAT/label/AAT.json",  # 修改为 AAT JSON 文件路径
    "./【AiDLab】A100/LAT/label/LAT.json"   # 修改为 LAT JSON 文件路径
]
dataset = FashionDataset(images_dirs, json_paths, transform=transform)

# 划分数据集为训练集、验证集和测试集，比例为 6:2:2
total_size = len(dataset)
train_size = int(0.6 * total_size)
val_size = int(0.2 * total_size)
test_size = total_size - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# 创建 DataLoader
trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valloader = DataLoader(val_dataset, batch_size=64, shuffle=False)
testloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 获取一个批次的数据
image_batch, label_batch = next(iter(trainloader))

# 随机选择一个索引
index = random.randint(0, len(image_batch) - 1)

# 显示图像和标签
imagedemo = image_batch[index]
imagedemolabel = label_batch[index]

# 转换为可显示格式
imagedemo = imagedemo.permute(1, 2, 0)  # 从 (C, H, W) 转换为 (H, W, C)

plt.imshow(imagedemo)
plt.axis('off')  # 关闭坐标轴

print(f'这张图片对应的标签是 {labellist[imagedemolabel]}')


In [ ]:
# 获取一个批次的数据
images, labels = next(iter(trainloader))

# 打印图像的形状
print(f"图像的形状: {images.shape}")

# 打印标签的内容
print(f"标签的内容: {labels}")



一个batchsize为64，3个通道（rgb），0123 分别是['shoes', 'clothing', 'bags', 'accessories']的索引

In [ ]:
# 定义模型的函数
def create_model(num_classes):
    model = models.resnet18(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, num_classes)  # 输出层与类别数相同
    return model

# 创建模型
model = create_model(len(labellist))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练循环
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    # 计算平均损失
    epoch_loss = running_loss / len(trainloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

    # 验证模型
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in valloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_accuracy = 100 * correct / total
    print(f"Validation Accuracy: {val_accuracy:.2f}%")

# 保存模型
torch.save(model.state_dict(), 'resnet_fashion_model.pth')
print("Model saved as resnet_fashion_model.pth")

# 测试模型
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

In [ ]:
import json
import os
from PIL import Image
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import torch
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from torch.optim.lr_scheduler import ReduceLROnPlateau

# 定义类别映射，将 AAT 和 LAT 的类别统一映射到四大类
category_mapping = {
    'Shoes': 'shoes',
    'Top': 'clothing',
    'Pants': 'clothing',
    'Skirt': 'clothing',
    'Dress': 'clothing',
    'Outwear': 'clothing',
    'Earing': 'accessories',
    'Bracelet': 'accessories',
    'Watches': 'accessories',
    'Hat': 'accessories',
    'Neckline': 'accessories',
    'Sunglasses': 'accessories',
    'Bags': 'bags',
    # AAT 数据集的类别映射（根据您的实际类别进行调整）
    'aat_category1': 'clothing',
    'aat_category2': 'shoes',
    # 添加更多的 AAT 类别映射
}

class FashionDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.images = [data['image'] for data in dataset]
        self.labels = [data['label'] for data in dataset]
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

def create_resnet18_model(num_classes):
    model = models.resnet18(pretrained=True)
    for param in model.parameters():
        param.requires_grad = True
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    return model

def train_model(model, train_loader, val_loader, criterion, optimizer, device, scheduler=None, num_epochs=50, patience=5):
    best_model_wts = model.state_dict()
    best_acc = 0.0
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    early_stopping_counter = 0
    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
                data_loader = train_loader
            else:
                model.eval()
                data_loader = val_loader

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in data_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(data_loader.dataset)
            epoch_acc = running_corrects.double() / len(data_loader.dataset)
            if phase == 'train':
                train_losses.append(epoch_loss)
                train_accuracies.append(epoch_acc.item())
            else:
                val_losses.append(epoch_loss)
                val_accuracies.append(epoch_acc.item())
                if scheduler is not None:
                    scheduler.step(epoch_loss)
                if epoch_loss < best_val_loss:
                    best_val_loss = epoch_loss
                    early_stopping_counter = 0
                    best_model_wts = model.state_dict()
                else:
                    early_stopping_counter += 1
                    if early_stopping_counter >= patience:
                        model.load_state_dict(best_model_wts)
                        plot_training_curves(train_losses, val_losses, train_accuracies, val_accuracies)
                        return model

    model.load_state_dict(best_model_wts)
    plot_training_curves(train_losses, val_losses, train_accuracies, val_accuracies)
    return model

def plot_training_curves(train_losses, val_losses, train_accuracies, val_accuracies):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, 'bo-', label='Training Loss')
    plt.plot(epochs, val_losses, 'ro-', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, 'bo-', label='Training Accuracy')
    plt.plot(epochs, val_accuracies, 'ro-', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.tight_layout()
    plt.show()

def test_model(model, test_loader, device):
    model.eval()
    running_corrects = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)
    total = len(test_loader.dataset)
    acc = running_corrects.double() / total
    print(f'Test Accuracy: {acc:.4f}')

def evaluate_model(model, test_loader, device, class_names):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    print("Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
    thresh = cm.max() / 2.
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            plt.text(j, i, format(cm[i, j], 'd'),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()

def predict(model, image_path, device, data_transforms, class_names):
    model.eval()
    image = Image.open(image_path).convert('RGB')
    image = data_transforms(image).unsqueeze(0)
    image = image.to(device)
    with torch.no_grad():
        outputs = model(image)
        _, preds = torch.max(outputs, 1)
    predicted_class = class_names[preds.item()]
    return predicted_class

def load_dataset(json_paths, image_dirs):
    dataset = []
    for json_path, image_dir in zip(json_paths, image_dirs):
        with open(json_path, 'r') as f:
            data = json.load(f)
        all_items = []
        for entry in data:
            all_items.extend(entry.get('question', []))
            all_items.extend(entry.get('answers', []))
        items = []
        for item in all_items:
            if 'AAT' in json_path:
                # 处理 AAT 数据集的标签格式
                category = item.split('/')[0]
                id_ = item.split('/')[-1]
                id_ = os.path.splitext(id_)[0]  # 去掉文件扩展名
            elif 'LAT' in json_path:
                # 处理 LAT 数据集的标签格式
                category, id_ = item.split('_')
            else:
                continue  # 如果有其他数据集，请根据其格式添加处理逻辑
            items.append({'category': category, 'id': id_})
        unique_items = { (item['category'], item['id']) for item in items }
        unique_items = [ {'category': cat, 'id': id_} for cat, id_ in unique_items ]
        for item in unique_items:
            category = item['category']
            id_ = item['id']
            label = category_mapping.get(category)
            if label is None:
                continue
            image_path = os.path.join(image_dir, f"{id_}.jpg")
            if not os.path.exists(image_path):
                continue
            image = Image.open(image_path).convert('RGB')
            dataset.append({'image': image, 'label': label})
    return dataset

def main():
    # 数据集的 JSON 文件和对应的图像目录
    json_paths = [
        './【AiDLab】A100/LAT/label/LAT.json',
        './【AiDLab】A100/AAT/label/AAT.json',
        # 添加更多数据集的 JSON 文件路径
    ]
    image_dirs = [
        './【AiDLab】A100/LAT/image',
        './【AiDLab】A100/AAT/image',
        # 添加更多数据集的图像目录
    ]

    # 加载并合并多个数据集
    dataset = load_dataset(json_paths, image_dirs)

    # 标签编码
    labels = [data['label'] for data in dataset]
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)
    class_names = label_encoder.classes_
    for idx, data_item in enumerate(dataset):
        data_item['label'] = encoded_labels[idx]

    # 数据增强和归一化
    data_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # 创建数据集和数据加载器
    full_dataset = FashionDataset(dataset, transform=data_transforms)
    train_size = int(0.6 * len(full_dataset))
    val_size = int(0.2 * len(full_dataset))
    test_size = len(full_dataset) - train_size - val_size
    train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

    batch_size = 8
    num_workers = 0
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    # 创建和训练模型
    num_classes = len(class_names)
    model = create_resnet18_model(num_classes=num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-5)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model = train_model(model, train_loader, val_loader, criterion, optimizer, device, scheduler=scheduler, num_epochs=50, patience=5)

    # 测试和评估模型
    test_model(model, test_loader, device)
    evaluate_model(model, test_loader, device, class_names)

    # 进行预测
    while True:
        image_path = input("请输入要预测的图像路径（输入'exit'退出）：")
        if image_path.lower() == 'exit':
            break
        if not os.path.exists(image_path):
            print("图像路径不存在，请重新输入。")
            continue
        predicted_class = predict(model, image_path, device, data_transforms, class_names)
        print(f"预测的类别是：{predicted_class}")

if __name__ == '__main__':
    main()
